## Setup and Import Libraries

In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [2]:
load_dotenv()

True

In [3]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [4]:
llm = ChatGroq(model="Gemma2-9b-It")

## Messages

In [5]:
llm.invoke(
    [
        HumanMessage(content="Hi, My name is Saad Tariq and I am a Data Scientist")
    ]
)

AIMessage(content="Hello Saad Tariq,\n\nIt's nice to meet you!  I'm glad you introduced yourself. \n\nWhat can I do for you today? Are you working on any interesting projects? Perhaps I can help with some data analysis or provide information on a specific topic related to data science.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 23, 'total_tokens': 87, 'completion_time': 0.116363636, 'prompt_time': 0.002112606, 'queue_time': 0.019646933000000002, 'total_time': 0.118476242}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-fb866c44-5cf2-4dcc-8a26-b54df77db3f0-0', usage_metadata={'input_tokens': 23, 'output_tokens': 64, 'total_tokens': 87})

In [6]:
llm.invoke(
    [
        HumanMessage(content="Hi, My name is Saad Tariq and I am a Data Scientist"),
        AIMessage(content="Hello Saad Tariq! It's nice to meet you.\n\nBeing a Data Scientist is a fascinating field. What kind of work are you involved in?"),
        HumanMessage(content="What is my name and What I do?")
    ]
)

AIMessage(content="You said your name is Saad Tariq and that you are a Data Scientist. \n\nIs there anything else you'd like to tell me about yourself or your work? I'm eager to learn more!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 74, 'total_tokens': 120, 'completion_time': 0.083636364, 'prompt_time': 0.004518702, 'queue_time': 0.150698336, 'total_time': 0.088155066}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1472e26d-1279-48bd-bc0f-7a837e498d8b-0', usage_metadata={'input_tokens': 74, 'output_tokens': 46, 'total_tokens': 120})

## Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of the inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input.

In [7]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
        
    return store[session_id]

In [8]:
config = {
    "configurable": {"session_id": "chat1"}
}

In [9]:
with_message_history = RunnableWithMessageHistory(llm, get_session_history)

In [10]:
response = with_message_history.invoke(
    [
        HumanMessage(content="Hi, My name is Saad Tariq and I am a Data Scientist"),
    ],
    config=config
)

In [11]:
response.content

"Hello Saad Tariq, it's nice to meet you! \n\nThat's great that you're a Data Scientist. It's a fascinating field with so many opportunities. \n\nWhat kind of data science work do you do? Are you working on any interesting projects right now?\n"

In [12]:
response = with_message_history.invoke(
    [
        HumanMessage(content="What is my name"),
    ],
    config=config
)

response.content

'Your name is Saad Tariq.  😊  \n\nI remembered from our earlier conversation.  Is there anything else I can help you with?\n'

## Changing the Config

In [13]:
config2 = {
    "configurable": {"session_id": "chat2"}
}

In [14]:
response = with_message_history.invoke(
    [
        HumanMessage(content="What is my name"),
    ],
    config=config2
)

response.content

'As an AI, I have no memory of past conversations and do not know your name. Can you tell me? 😊\n'

In [15]:
response = with_message_history.invoke(
    [
        HumanMessage(content="Hi, My name is Shawn"),
    ],
    config=config2
)

response.content

"Hi Shawn, it's nice to meet you! 👋 \n\nIs there anything I can help you with today?\n"

In [16]:
response = with_message_history.invoke(
    [
        HumanMessage(content="What is my name"),
    ],
    config=config2
)

response.content

'Your name is Shawn. 😊  \n\nI remembered!  Do you need me to remember anything else?\n'